# ROOT analyser

In [ ]:
import fitz
import cv2 as cv
import numpy as np
import json

from src.viz.graphs import *
from src.viz.images import *
from src.utils import *
from src.file_reading import *
from src.detection.elements import *
from src.getFirstFrame import *

from sklearn.mixture import GaussianMixture,BayesianGaussianMixture

In [ ]:
large_imshow = lambda img: scaled_imshow(img,fx=0.8,fy=0.8)
simshow = lambda img: scaled_imshow(img,fx=0.3,fy=0.3)
mini_imshow = lambda img: scaled_imshow(img,fx=0.2,fy=0.2)

## Data

The input data is be divided into 3 groups depending on the difficulty for example:
- easy: perfect top down view, the game elements are not covered with hands when carrying them, the lighting is good
- medium: strong light at the side causing shadows,
- difficult: same as medium + a slightly angled camera, hands covering the pieces

There are 3 clips per difficulty. The data is located in a [google drive]((https://drive.google.com/drive/folders/1VrQ98TC5jPmWk1QYr3lUP3SGk_3_AEmx?usp=sharing))


In [ ]:
data_dir = ".\\data\\"
clip_dirs = {"easy": data_dir+'\\easy\\',
            "medium": data_dir+'\\medium\\',
            "hard": data_dir+'\\hard\\'}

In [ ]:
first_frames = dict([(diff,getFirstFrame(_dir+"clip_0.mp4")) for diff,_dir in clip_dirs.items()])

In [ ]:
mini_imshow(first_frames["easy"])

In [ ]:
mini_imshow(first_frames["medium"])

In [ ]:
mini_imshow(first_frames["hard"]) # TODO rotate hard clips

The game is played between 2 factions: Eyrie Dynasties (blue birds), Marquise de Cat (orange cats). The board is a Winter Map. Because the clearings in the forest are barely differentiable, a mask was created to help with detecting static elements of the board. 

In [ ]:
game_data_dir = data_dir+"game_data\\"

In [ ]:
board_mask = cv.imread(game_data_dir+"board_mask.png")

simshow(board_mask)

- The red indicates the where the score track is. 
- The green defines where craftable items are.
- The blue shows where the clearing approximately are, with the black squares showing where building spaces are.

JSON was created to define paths on the map, done purely for drawing a graph of the map.

In [ ]:
with open("./data/game_data/board_info.json", "r") as info_file:
    board_info = json.load(info_file)

    draw_map(board_info)

To help with detection a print and play set is used with all the elements taken from [PnP PARADISE](https://www.pnpparadise.com/set1/root).

In [ ]:
board_ref = read_pdf(data_dir+'game_data\\board.pdf')

simshow(board_ref)

## Milestone 1

In this phase, the following things were detected:
- the black dice tray along with the dice on it
- the board

### Dice tray detection

In [ ]:
img = cv.imread('.\\data\\raw\\first_frame.png')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [ ]:
mini_imshow(img)

The dice tray is all black so a simple threshold was performed.

In [ ]:
tray_cont,dice1_cont,dice2_cont,img_cont = detect_dice_tray(img)
mini_imshow(img_cont)

In [ ]:
tray = crop_contour(img,tray_cont)

dice1 = crop_contour(img,dice1_cont)
dice2 = crop_contour(img,dice2_cont)

mini_imshow(tray)
imshow(dice1)
imshow(dice2)

### Board detection

Detecting the board was harder as it has much more details.

In [ ]:
board_ref = read_pdf('.\\data\\game_data\\board.pdf')
board_gray = cv.GaussianBlur(cv.cvtColor(board_ref, cv.COLOR_BGR2GRAY),(7,7),0)

simshow(board_ref)

To achieve this steps descriptors are used, in particular the SIFT detector. To quickly match the descriptors FLANN algorithm is used.

In [ ]:
M,drawn_matches,contour = detect_board(img,board_ref)

In [ ]:
simshow(drawn_matches)

In [ ]:
simshow(crop_contour(img,contour))

This code doesn't have to be run much, because the board should not a lot move in the clips

In the milestone 1, there were also attempts to segment the image using a Gaussian Mixture, but they were quite slow and not effective